In [ ]:
##in this version i will say screw pipelines and just do it without for now ... 
#maybe I will see what this infinity issue is if I can inspect the resuly of each step manually

In [ ]:
##Round 1 cleaning complete. v1 decisions: keep all nan as new feature/value; no skew or outlier action has been taken
##Round 1 exploration complete. (a) Distribution of each feature explored (b) feature-target and feature-feature relationships explored 

##Round 1 modelling Plan: make one model for each notebook, try to do systematic tuning ... take best few models (?) or is it too soon?
##then go back and do feature engineering and feature selection based on these base-line models
##Build pipe-line: dummies, scaling, tuning ...

##next steps:
##
##Flask API Deployment
##build small webpage interaction


### Tuning of KNN 

Main parameters: k = num_neighbors, and metric

In [ ]:
## following https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

In [ ]:
##brief review of KNN vs K-means ... 
##K-means (unspervised) has no labels, just feature space ... you create k-labels (choice of k is tricky and important)
##then run thru k-means clustering algorithm to assign k-labels (clusters) to the data 
##b
##KNN (supervised) is simpler because data already comes with p-labels (the role of k is different here) 
##in the training data on the test data you simply look at the k-nearet neighors (of training set) and take majority vote 
##on what label to assign to new point. The choice of k is important and the metric can change the outcomes.

In [40]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer, OrdinalEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA 
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score
import seaborn as sns

import pickle

#sns.set()
sns.set(rc={'figure.figsize':(8,5)})



In [41]:
## cleaning tool


def perc_null(X):
    
    total = X.isnull().sum().sort_values(ascending=False)
    data_types = X.dtypes
    percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)

    missing_data = pd.concat([total, data_types, percent], axis=1, keys=['Total','Type' ,'Percent'])
    return missing_data


In [42]:
path = "/Users/joejohns/Downloads/data_loans.csv"

data = pd.read_csv(path) 
print(data.shape)
data.head(3)

(614, 13)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y


## data processing from round 1


In [43]:
perc_null(data).index

    

Index(['Credit_History', 'Self_Employed', 'LoanAmount', 'Dependents',
       'Loan_Amount_Term', 'Gender', 'Married', 'Loan_ID', 'Education',
       'ApplicantIncome', 'CoapplicantIncome', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [44]:
data.rename(columns = {'ApplicantIncome':'Applicant_Income', 'CoapplicantIncome': 'Coapplicant_Income', 'LoanAmount':'Loan_Amount'}, inplace = True)

In [45]:
target = ['Loan_Status']
ID_feats = ['Loan_ID',] 

#feature types:

num_feats = ['Applicant_Income', 'Coapplicant_Income', 'Loan_Amount', 'Loan_Amount_Term',]

ord_cat_feats = [ 'Dependents', 'Education', ]
nom_cat_feats = ['Gender', 'Married', 'Self_Employed', 'Credit_History', 'Property_Area',]



In [46]:
##fix ordinal cats (write general function which does this, no ordinal encoder)
for item in ['0', '1', '2', '3+']:
    if item != '3+':
        i = int(item)
    else:
        i = 3
    data.loc[data['Dependents'] == item,'Dependents']  = i


In [53]:
#fix target; make it numeric; useful for regression modelling

data.loc[data["Loan_Status"] == 'Y', "Loan_Status"] = 1
data.loc[data["Loan_Status"] == 'N', "Loan_Status"] = 0



In [54]:
#fix this nom cat ... float --> string

data.loc[data['Credit_History'] ==0.0, 'Credit_History'] = 'No'
data.loc[data['Credit_History'] == 1.0, 'Credit_History'] = 'Yes'  #I use capitalized "Yes", "No" to be conistent with their other entries

In [47]:
#turn this into ordinal cat

data.loc[data['Education']=='Not Graduate', "Education"] = 0
data.loc[data['Education']=='Graduate', "Education"] = 1

In [49]:

#create one new categorical for both (if either or both is unknown)
data['missing_data'] = 'No'  
for feature in num_ord_nan:
    data.loc[(data[feature].isnull()) , 'missing_data'] = 'Yes'

    


In [55]:
#update nom_cat_feats (no null values, ok)

nom_cat_feats+= ['missing_data']

In [48]:
#set up some feature lists for imputing and scaling 

num_ord_feats = ord_cat_feats+num_feats

#can make a for loop that extracts these ... 
num_ord_nan_new_cat = [ 'Loan_Amount', 'Loan_Amount_Term', 'Dependents', ] #make new category for these 'unk' = yes/no; ##impute by mean
nom_cat_nan_dummies = ['Credit_History', 'Self_Employed', 'Gender', 'Married',]  ##make new dumies cat_val for nan
nom_cat_dummies = ['Property_Area',]  #regular dummies




#### Done with cleaning categorical features ... 
note: we have made preliminary decision to make all NaN in categorical columns into new cat_val ... \
we'll see how this works out ... in some cases it seems to be a good predictor.

In phase 2 investigation, below, we will investigate the naive predictive strength of all the \
categorical features (including NaN as 'unk')


In [ ]:
##baseline models
from sklearn.model_selection import train_test_split # For train/test splits
from sklearn.neighbors import KNeighborsClassifier # The k-nearest neighbor classifier
from sklearn.feature_selection import VarianceThreshold # Feature selector
from sklearn.pipeline import Pipeline # For setting up pipeline
# Various pre-processing steps
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler, LabelEncoder

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import GridSearchCV # For optimization


In [ ]:
pd.get_dummies(data, columns = nom_cat_feats)

In [35]:

class DataframeTransformer():
    def __init__(self, func):
        self.func = func

    def transform(self, input_df, **transform_params):
        return self.func(input_df)

    def fit(self, X, y=None, **fit_params):
        return self

# this function takes a dataframe as input and
# returns a modified version thereof
def get_dummies_dataframe(input_df):
    return pd.get_dummies(input_df, columns = nom_cat_feats)


# this pipeline has a single step
pipeline = Pipeline([
    ("get_dummies", DataframeTransformer(get_dummies_dataframe))
])

In [39]:

freq_imp = SimpleImputer(missing_values=np.nan,strategy="most_frequent")
mean_imp = SimpleImputer(missing_values=np.nan, strategy="mean")
scaler = StandardScaler()
dummies = OneHotEncoder(handle_unknown='ignore')

#other options:
#1. create new class that has transform and fit and uses make_dummies instead 
#2. put a FunctionTransformer() in the pipeline 

#imputing

impute_mean_feats = ['Loan_Amount',]
impute_freq_feats = [ 'Loan_Amount_Term', 'Dependents']


#scaling
scaling_feats = ord_cat_feats+num_feats

#dummies
dummies_feats = nom_cat_feats #default setting of this variable ... 
#ordinal_encode
ord_feats = ord_cat_feats  #done manually at this point

preprocess = ColumnTransformer([
    ('most_freq_imp', freq_imp, impute_freq_feats),
    ('mean_imp', mean_imp, impute_mean_feats), #only one numerical feature has NaN right now
  #  ('scaler', scaler, scaling_feats),
   # ('dummies', dummies, dummies_feats), 
])



pipeline = Pipeline([
    ('preprocess', preprocess),
    ('knn', KNeighborsClassifier())
])

NameError: name 'KNeighborsClassifier' is not defined

In [ ]:
freq_imp.fit_transform(data.loc[:, impute_freq_feats])

In [ ]:
#let me now follow https://machinelearningmastery.com/modeling-pipeline-optimization-with-scikit-learn/
#which does cv = 3 instead of cv = stratififedkfold( ... )

model_feats = ord_cat_feats+nom_cat_feats+num_feats

y = data.loc[:, target]
X= data.loc[:, model_feats]

# Split the data into test and train
X_train, X_test, y_train, y_test = train_test_split(
    X,  
    y, 
    test_size=1/3,
    random_state=0)
 
print(X_train.shape)
print(X_test.shape)

#np.array(X)

In [ ]:
print(target)

In [ ]:
print(model_feats)

In [ ]:
perc_null(X_train)

In [ ]:
preprocess.fit(data)

In [ ]:
X_train_prep = preprocess.fit_transform(X_train)

In [ ]:
X_train_prep  #hmm ColumnTransformer returns an array ... 

In [ ]:
knn = KNeighborsClassifier()

knn.fit(X_train_prep, y_train)

In [ ]:

knn = pipeline.fit(X_train, y_train)
print('Training set score: ' + str(knn.score(X_train,y_train)))
print('Test set score: ' + str(knn.score(X_test,y_test)))


In [ ]:
what ... the fuck... it seems there is an infinite value (or too large)